In [538]:
import pandas as pd

In [539]:
materials = pd.read_csv("materials.csv");

In [540]:
materials.head(5)

,tag,category,cutting-power,density,cutting-protection,blunt-protection,penentration-protection,magic,name
0,arrow-bone,bow-ammo,1.0,0.2,0.0,0.0,0.0,0.0,Bone arrow
1,arrow-zaz,bow-ammo,2.0,0.2,0.0,0.0,0.0,1.0,Zaz arrow
2,cotton,plant,0.0,0.1,0.2,0.2,0.2,0.1,Cotton
3,textile,material,0.0,1.0,1.0,1.0,1.0,0.1,Textile
4,small-bone-rat,bone,1.0,0.5,0.5,0.5,0.5,0.0,"Bone(rat, small)"


In [541]:
materials.axes

[RangeIndex(start=0, stop=31, step=1),
 Index(['tag', 'category', 'cutting-power', 'density', 'cutting-protection',
        'blunt-protection', 'penentration-protection', 'magic', 'name'],
       dtype='object')]

In [542]:
number_columns = ['cutting-power', 'density', 'cutting-protection',
        'blunt-protection', 'penentration-protection', 'magic']
strings = [
    'tag',
]

## Generating enum

In [543]:
main_string = ""
configuration_string = ""
accessor_string = ""

### Enums for categorical data:

In [544]:
ENUM_MATERIAL = "MATERIAL"
ENUM_MATERIAL_PLURAL = "MATERIALS"
ENUM_CATEGORY = "MATERIAL_CATEGORY"
ENUM_CATEGORY_PLURAL = "MATERIAL_CATEGORIES"

In [545]:
def enum_name(s: str):
    return s.replace("-", "_").upper()

def method_name(s: str):
    return s.replace("-", "_")

def getter_name(s: str):
    return "get " + method_name(s) + "()"

def new_line(s:str):
    return s + "\n"

def comma(s: str):
    return s + ','

def space(s: str):
    return s + ' '

def quotes(s: str):
    return f'"{s}"'

def tab(s: str):
    return "    " + s

def collection_name(singular: str):
    def _collection_name(s: str):
        return singular + '.' + s
    return _collection_name

def accessor(array_name: str, property_name: str):
    return (
        f'    {getter_name(property_name)} {{\n'
        f'        return MaterialConfiguration.{array_name}[this._id]\n'
        f'    }}\n'
    )

def categorical_to_enum(s: pd.Series, singular: str, plural: str, axe):
    main_string = (
        f'const enum {singular} ' '{\n'
        f'{s.apply(enum_name).drop_duplicates(keep="first").apply(comma).apply(new_line).apply(tab).sum()}'
        '}\n'
    )
    array = f'    static {plural}: {singular}[] = [{s.apply(enum_name).drop_duplicates(keep="first").apply(collection_name(singular)).apply(comma).apply(space).sum()}]\n'
    return main_string, array

In [546]:
enum, array = categorical_to_enum(materials["tag"], ENUM_MATERIAL, ENUM_MATERIAL_PLURAL, "tag")
configuration_string += array
main_string += enum

enum, array = categorical_to_enum(materials["category"], ENUM_CATEGORY, ENUM_CATEGORY_PLURAL, "category")
configuration_string += array
main_string += enum

accessor_string += accessor(f'{enum_name(ENUM_MATERIAL)}_STRING', "tag")
accessor_string += accessor(f'{enum_name(ENUM_CATEGORY)}_STRING', "category")

accessor_string += accessor(f'{enum_name(ENUM_MATERIAL_PLURAL)}', "id")
accessor_string += accessor(f'{enum_name(ENUM_CATEGORY_PLURAL)}', "CATEGORY")

configuration_string += f'    static {enum_name(ENUM_MATERIAL)}_STRING : Record<{ENUM_MATERIAL}, string> = [{materials["tag"].apply(str).apply(quotes).apply(comma).apply(space).sum()}]\n'
configuration_string += f'    static {enum_name(ENUM_CATEGORY)}_STRING : Record<{ENUM_MATERIAL}, string> = [{materials["category"].apply(str).apply(quotes).apply(comma).apply(space).sum()}]'

### Data

In [547]:
def number_series_to_array(s: pd.Series, axis_name) -> str:
    return f'    static {ENUM_MATERIAL}_{enum_name(axis_name)} : Record<{ENUM_MATERIAL}, number> = [{s.apply(str).apply(comma).apply(space).sum()}]'


In [548]:
for item in number_columns:
    configuration_string = "\n".join([configuration_string, number_series_to_array(materials[item], item)])
    accessor_string = "".join([accessor_string, accessor(f'{ENUM_MATERIAL}_{enum_name(item)}', item)])

### Fat material

In [549]:
main_string = "\n".join([main_string, """


interface MaterialData {
    readonly id: MATERIAL
    readonly CATEGORY: MATERIAL_CATEGORY
    readonly tag: string
    readonly category: string
    readonly cutting_power: number
    readonly density: number
    readonly cutting_protection: number
    readonly blunt_protection: number
    readonly penentration_protection: number
    readonly magic: number
}
"""])

In [550]:
configuration_string = (
    f'export class MaterialConfiguration {{\n'
    f'    private static _instance: MaterialConfiguration;\n'
    f'{configuration_string}\n'
    # f'    get_instance() {{\n'
    # f'        if (MaterialConfiguration._instance) return MaterialConfiguration._instance;\n'
    # f'        MaterialConfiguration._instance = new MaterialConfiguration()\n'
    # f'    }}\n'
    f'}}\n'
)

In [551]:

accessor_string = (
    f'\n'
    f'export class FatMaterial implements MaterialData {{\n'
    f'    private _id: MATERIAL\n'
    f'    constructor(id: MATERIAL) {{\n'
    f'        if (!(id in MaterialConfiguration.MATERIAL_STRING)) {{\n'
    f'            throw new Error(`Invalid material id: ${id}`);\n'
    f'        }}\n'
    f'        this._id = id\n'
    f'    }}\n'
    f'{accessor_string}'
    f'}}\n'
)

print(main_string +  configuration_string + accessor_string, file=open("materials.ts", "w"))